In [2]:
import pandas as pd
import numpy as np

input_data = pd.read_csv("Global_Superstore2.csv",encoding='ISO-8859-1')
# input_data['Order Date'] = pd.to_datetime(input_data['Order Date'])

input_data['Order Date'] = [ i.split('-') for i in input_data['Order Date'] ]
input_data['Date'] = [ int(i[2]+i[1]) for i in input_data['Order Date'] ]


### 計算 Recency

In [24]:
recency_temp = input_data[['Customer ID','Date']].drop_duplicates()
temp = []
for i in recency_temp['Date']:
    # print(i)
    if i >= 201401:
        temp.append(4)
    elif i < 201401 and i >= 201301:
        temp.append(3)
    elif i < 201301 and i >= 201201:
        temp.append(2)
    elif i < 201201 and i >= 201101:
        temp.append(1)
    else:
        temp.append(0)
recency_temp['recency'] = temp
recency = recency_temp.groupby('Customer ID',as_index=False)['recency'].max()
recency

,Customer ID,recency
0,AA-10315,4
1,AA-10375,4
2,AA-10480,4
3,AA-10645,4
4,AA-315,4
...,...,...
1585,YS-21880,4
1586,ZC-11910,4
1587,ZC-21910,4
1588,ZD-11925,4


#### 計算 Frequency

In [23]:
frequency_temp = input_data[['Customer ID' , 'Order ID']].drop_duplicates()
freq_count = frequency_temp.groupby('Customer ID',as_index=False)['Order ID'].aggregate('count').reset_index().sort_values('Order ID', ascending=False, axis=0)

freq_block = freq_count.loc[:,['Order ID']]
freq_block['block'] = pd.qcut(freq_block['Order ID'] , 4)
freq_block = freq_block[['block']].drop_duplicates()
freq_block


,block
1206,"(26.0, 41.0]"
1076,"(14.0, 26.0]"
1238,"(6.0, 14.0]"
1584,"(0.999, 6.0]"


In [26]:
temp = []
for i in freq_count['Order ID']:
    if i >= 26:
        temp.append(4)
    elif i < 26 and i >= 14:
        temp.append(3)
    elif i < 14 and i >= 6:
        temp.append(2)
    else:
        temp.append(1)

frequency = freq_count.loc[:,['Customer ID']]
frequency['frequency'] = temp
frequency

,Customer ID,frequency
1206,PO-18850,4
862,KH-16690,4
1067,MP-17965,4
294,CK-12205,4
1579,WB-21850,4
...,...,...
1016,MG-7875,1
1001,ME-8010,1
850,KD-6270,1
1260,RC-9825,1


### 計算 monetary

In [31]:
monetary_temp = input_data.groupby('Customer ID')['Sales'].aggregate('sum').reset_index().sort_values('Sales',ascending=False, axis=0)
monetary_temp

uni_sales = monetary_temp.loc[:,['Sales']].drop_duplicates()
uni_sales['block'] = pd.qcut(uni_sales['Sales'],4)
uni_sales = uni_sales[['block']].drop_duplicates()
uni_sales

,block
1463,"(13133.098, 35668.121]"
907,"(6248.136, 13133.098]"
889,"(1674.812, 6248.136]"
810,"(7.172, 1674.812]"


In [32]:
temp = []
for i in monetary_temp['Sales']:
    if i >= 13134:
        temp.append(4)
    elif i < 13134 and i >= 6249:
        temp.append(3)
    elif i < 6249 and i >= 1675:
        temp.append(2)
    else:
        temp.append(1)

monetary = monetary_temp.loc[:,['Customer ID']]
monetary['monetary'] = temp
monetary


,Customer ID,monetary
1463,TA-21385,4
623,GT-14710,4
1489,TC-20980,4
1405,SM-20320,4
237,BW-11110,4
...,...,...
1069,MP-7470,1
143,BD-1500,1
1260,RC-9825,1
1017,MG-7890,1


In [33]:
customer_analyz = pd.merge(recency,frequency)
customer_analyz = pd.merge(customer_analyz,monetary)
customer_analyz

,Customer ID,recency,frequency,monetary
0,AA-10315,4,3,4
1,AA-10375,4,3,2
2,AA-10480,4,3,4
3,AA-10645,4,4,4
4,AA-315,4,2,2
...,...,...,...,...
1585,YS-21880,4,4,4
1586,ZC-11910,4,1,1
1587,ZC-21910,4,4,4
1588,ZD-11925,4,2,2
